## Configure the Environment

Import FABRIC and Paramiko packages, define program constants, and declare certain variables for keys and tokens

In [ ]:
import os
import time
import json

from fabrictestbed.slice_manager import SliceManager, Status

import paramiko

slice_id=None  # Set to your slice ID if you already have a slice

VERBOSE=False  # Set to true for more output
CLEANUP=False  # Set to true to be able to delete nodes in cleanup section

fabric_refresh_token=None
ssh_key = None

### Save the Initial Refresh Token

In [ ]:
if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']

%store fabric_refresh_token

if VERBOSE:
    print("Refresh Token: {}".format(fabric_refresh_token))

### Create Slice Manager

The slice manager allows for us to reserve and access physical resources from FABRIC.

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

# Create Slice Manager
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, 
                             refresh_token=fabric_refresh_token, project_name='all', scope='all')

### Refresh the ID Token

In [ ]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
if VERBOSE:
    print("New Refresh Token: {}".format(fabric_refresh_token))
    print("Stored new Refresh Token")
    %store fabric_refresh_token

### Configure SSH Key

We need this key so that we can access our physical nodes at FABRIC.

In [ ]:
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

## Provision Resources

### Provision/Query Slice

If no slice ID was defined by the user, this cell will create a new slice and obtain its ID. Once an ID is obtained, we will run a slice manager checking the given slice's status until it is stable for us to work with.

In [ ]:
if slice_id == None:
    from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, LinkType, Layer
    
    # Create Experiment Topology
    experiemnt = ExperimentTopology()
    slice_name="ParamikoSlice"
    
    # Add Node1 with: GPU: 1xSharedConnectX-6, 1xRTX6000, 1xP4510
    node1 = experiemnt.add_node(name='node1', site='RENC')

    node1_capacity = Capacities()
    node1_capacity.set_fields(core=2, ram=16, disk=100)
    node1.set_properties(capacities=node1_capacity, image_type='qcow2', image_ref='default_ubuntu_20')

    node1_nvme = node1.add_component(ctype=ComponentType.NVME, model='P4510', name='n1_nvme')
    node1_gpu  = node1.add_component(ctype=ComponentType.GPU, model='RTX6000', name='n1_gpu')
    node1_nic = node1.add_component(ctype=ComponentType.SharedNIC, model='ConnectX-6', name='n1_nic')
    
    slice_graph = experiemnt.serialize()

    # Request slice from Orchestrator
    status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key)

    if VERBOSE:
        print("Request {}".format(status))
    slice_id=reservations[0].slice_id

    if VERBOSE:
        print("Reservations: {}".format(reservations))
        print("Slice ID: {}".format(slice_id))
    
status, slices = slice_manager.slices()
    
status, slice_status = slice_manager.slice_status(slice_id=slice_id)
slice_state = slice_status.slice_state
while slice_state != 'StableOK':
    status, slice_status = slice_manager.slice_status(slice_id=slice_id)
    slice_state = slice_status.slice_state
    if VERBOSE:
        print("Response Status {}".format(status))
        print("Slice State: {}".format(slices[0].slice_state))
    time.sleep(20)
    
if VERBOSE:
    print("Response Status {}".format(status))
    print("Slice Status {}".format(slice_status))

### Query Slivers

This cell accesses the slivers (our physical nodes) from the slice we have defined above.

In [ ]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

for sliver in slivers:
    sliver_id=sliver.reservation_id
    status, reservation_status = slice_manager.sliver_status(slice_id=slice_id, sliver_id=sliver_id)

    if VERBOSE:
        print("Response Status {}".format(status))
        print("Reservation Status {}".format(reservation_status))

## SSH to Slivers

### Configure Script

This string represents a configurable bash script that we will run remotely in each of our nodes.

In [ ]:
script= '#!/bin/bash   \n' \
        'echo "Hello World!"   \n'

## Access Slivers

Using the Paramiko Python package, we can access and run commands through each of our physical nodes at FABRIC.

In [ ]:
key = paramiko.RSAKey.from_private_key_file("/home/fabric/.ssh/id_rsa")
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

for sliver in slivers:
    node_name = sliver.name
    management_ip = sliver.management_ip
    
    print("Node {0} IP {1}".format(node_name, management_ip))
    
    client.connect(management_ip,username='ubuntu',pkey = key)

    stdin, stdout, stderr = client.exec_command('echo \"' + script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
    print ('')
    print (str(stdout.read(),'utf-8').replace('\\n','\n'))

    client.close()

## Clean Up

### Delete Slice

If the `CLEANUP` variable has been set to **true** at the top of this notebook, then the slice will be deleted via its IP.

In [ ]:
if CLEANUP:
    status, result = slice_manager.delete(slice_id=slice_id)

    if VERBOSE:
        print("Response Status {}".format(status))
        print("Response received {}".format(result))